<a href="https://colab.research.google.com/github/thanhnhan311201/CS114.L11.KHCL/blob/master/DemoSarcasmLogisticsRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/thanhnhan311201/CS114.L11.KHCL

fatal: destination path 'CS114.L11.KHCL' already exists and is not an empty directory.


In [ ]:
!pip install stop-words

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import json
import string
from collections import Counter

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_json("/content/CS114.L11.KHCL/LogisticsRegression/ThucHanh/Sarcasm_Project/Sarcasm_Headlines_Dataset.json", lines=True)
data.head()
del data['article_link']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data['headline'], data['is_sarcastic'], test_size=0.33)
print(X_train.shape, X_test.shape)

(17895,) (8814,)


In [ ]:
ListStopWord = set(stopwords.words('english'))
ListPunchLine = list(string.punctuation)
print(ListStopWord)
print(ListPunchLine)

{'am', 'some', 've', 'herself', "wasn't", 'why', 'out', 'off', 'your', 'only', 'to', 'other', 'is', 'into', 'didn', 'these', 'at', 'from', 'yourselves', 'ours', 'just', 'for', 'where', "didn't", 'needn', 't', "you'd", 'and', 'by', "isn't", 'should', 'were', 'against', 'if', 'any', "haven't", "it's", 'itself', 'because', "hasn't", 'not', 'our', 'the', 'each', 'once', "wouldn't", 'his', 'there', 'of', 'will', 'few', 'won', "needn't", 'in', "mightn't", 'down', 'during', 'an', 'weren', 'theirs', 'above', 'has', 'its', 'shouldn', "you've", 'over', 'further', 'did', 'who', 'too', 'was', 'whom', 'while', 'same', 'she', 'so', 'before', 'most', 'been', 'with', 'd', 'shan', "aren't", 'ain', "don't", "you're", 'himself', 'my', 'don', 'can', 'him', 'themselves', 'between', "should've", 'those', 'more', 'through', "shan't", 'their', "weren't", 'hasn', 'll', 'ourselves', 'couldn', 'had', 'both', 'or', 'me', 'after', 'myself', 's', 'now', 'does', 'what', 'that', 'i', 'it', 'on', 'do', 'wasn', 'haven'

In [ ]:
def Normalize(text):
    ans = []
    for headline in list(text):
        for p in ListPunchLine:
            if p in headline:
                headline = headline.replace(p,'')
        new_headline = []
        for i in headline.split():
            if i.strip().lower() not in ListStopWord:
                new_headline.append(i.strip().lower())
        headline = " ".join(new_headline)
        ans.append(headline)
    return pd.Series(ans)

In [ ]:
print(X_train)

19801    mike pence to anti-abortion crowd: trump's sup...
18378                         reality show slowly sinks in
1611     hey, taylor swift, caitlyn jenner got a call f...
432                     sheets changed after every breakup
21411    a transgender student who was reportedly banne...
                               ...                        
25402         rupaul is getting a star on the walk of fame
18863             composer dan licht on writing for dexter
16943    perfect response to heckler somewhere in prop ...
11755              local band cleverly alters product logo
21809          bus driver appears to have had rough summer
Name: headline, Length: 17895, dtype: object


In [ ]:
X_train = Normalize(X_train)
print(X_train.size)
print(X_train)

17895
0        mike pence antiabortion crowd trumps supreme c...
1                                reality show slowly sinks
2           hey taylor swift caitlyn jenner got call kanye
3                             sheets changed every breakup
4        transgender student reportedly banned school r...
                               ...                        
17890                        rupaul getting star walk fame
17891                    composer dan licht writing dexter
17892    perfect response heckler somewhere prop comedi...
17893              local band cleverly alters product logo
17894                      bus driver appears rough summer
Length: 17895, dtype: object


In [ ]:
print(X_train[0])

mike pence antiabortion crowd trumps supreme court pick mold antonin scalia


In [ ]:
def CreateListWord(ListHeadLine):
    all_words = []
    for headline in ListHeadLine:
        all_words.extend(headline.split())
    return all_words

In [ ]:
def WordToVector(text, F):
    X_counts = pd.Series(Counter(CreateListWord(pd.Series(text))))    
    new = X_counts.reindex(F.index, fill_value= 0)
    return list(new) 

In [ ]:
def CreateVectorTable(X, mask):
    ans = []
    for headline in X:
        ans.append(WordToVector(headline, mask))
    ans = np.array(ans)
    return ans

In [ ]:
mask = pd.Series(Counter(CreateListWord(X_train)))
ans = CreateVectorTable(X_train, mask)
print(ans)

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 1]]


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5,max_iter=100000)

In [ ]:
logreg.fit(ans, Y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100000, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [ ]:
X_test = Normalize(X_test)

In [ ]:
X_Test_Vector = CreateVectorTable(X_test, mask)

In [ ]:
print(X_Test_Vector)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
logreg.score(X_Test_Vector, Y_test)

0.7607215793056501

In [ ]:
data = pd.read_json("/content/CS114.L11.KHCL/LogisticsRegression/ThucHanh/Sarcasm_Project/Sarcasm_Headlines_Dataset_v2.json", lines=True)

In [ ]:
del data['article_link']

In [ ]:
X_test1, X_test2, Y_test1, Y_test2 = train_test_split(data['headline'], data['is_sarcastic'], test_size= 0.5)

In [ ]:
X_test1 = Normalize(X_test1)

X_test1 = CreateVectorTable(X_test1, mask)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
Y_test1 = list(Y_test1)

In [ ]:
logreg.score(X_test1, Y_test1)

0.9046753791320148

In [ ]:
X_test2 = CreateVectorTable(X_test2, mask)

In [ ]:
Y_test2 = list(Y_test2)
logreg.score(X_test2, Y_test2)

0.9350104821802935